# Train GPT-OSS 20B → Rust Coding Agent

End-to-end pipeline for training a Rust coding agent on top of OpenAI's GPT-OSS 20B (MoE, ~3.6B active params).

**4-Phase Pipeline:**
1. **Lang Adapter** — Rust domain specialisation via QLoRA (script 13 + 19)
2. **Core Agent SFT** — Agent trajectory training with tool use (script 14)
3. **IPO Preference** — Identity Preference Optimisation on ranked pairs (script 17)
4. **GRPO RL** — Group Relative Policy Optimisation with execution rewards (script 18)

**Requirements:**
- **GPU**: A100 40GB+ (80GB recommended for full context lengths)
- **Storage**: Google Drive for persistent checkpoints
- **Rust toolchain**: Installed automatically (rustup + cargo-mutants)

---
## Step 0: Environment Setup

In [ ]:
#@title ### 0.1 Mount Google Drive & Clone Repository
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted")
else:
    print("Running locally")

REPO_URL = "https://github.com/rmarnold/llm-training-pipeline.git"  #@param {type:"string"}
BRANCH = "main"  #@param {type:"string"}

REPO_DIR = "/content/llm-training-pipeline"

if IN_COLAB:
    if os.path.exists(REPO_DIR):
        %cd {REPO_DIR}
        !git pull origin {BRANCH}
    else:
        !git clone -b {BRANCH} {REPO_URL} {REPO_DIR}
        %cd {REPO_DIR}

    PROJECT_ROOT = REPO_DIR
else:
    PROJECT_ROOT = os.getcwd()

os.chdir(PROJECT_ROOT)
print(f"\nProject root: {PROJECT_ROOT}")

In [ ]:
#@title ### 0.2 Install Dependencies
#@markdown Installs pipeline deps, Unsloth (Colab-optimised), and the Rust toolchain.

if IN_COLAB:
    print("Installing Python dependencies...")
    print("=" * 60)
    !pip install -q -e ".[gpt_oss,rust_eval,colab]"
    !pip install -q "unsloth[colab-new]"
    !pip install -q flash-attn --no-build-isolation 2>/dev/null || true

    print("\nInstalling Rust toolchain...")
    print("=" * 60)
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"
    !cargo install cargo-mutants

    # Verification
    print("\n" + "=" * 60)
    print("Dependency Verification:")
    print("=" * 60)

    for pkg in ["unsloth", "trl", "peft", "datasets", "tiktoken"]:
        try:
            mod = __import__(pkg)
            ver = getattr(mod, "__version__", "OK")
            print(f"\u2713 {pkg}: {ver}")
        except ImportError as e:
            print(f"\u2717 {pkg}: {e}")

    import subprocess
    for cmd, label in [("cargo --version", "cargo"), ("cargo-mutants --version", "cargo-mutants")]:
        result = subprocess.run(cmd.split(), capture_output=True, text=True)
        if result.returncode == 0:
            print(f"\u2713 {label}: {result.stdout.strip()}")
        else:
            print(f"\u2717 {label}: not found")

    print("=" * 60)
else:
    print("Running locally \u2014 ensure deps are installed: pip install -e '.[gpt_oss,rust_eval]'")

In [ ]:
#@title ### 0.3 Configure Pipeline { run: "auto" }
#@markdown ---
#@markdown ### Training Scope

training_scope = "full"  #@param ["full", "quick_test", "lang_adapter_only", "skip_to_rl"]
#@markdown **Scopes:**
#@markdown - `full`: All 4 phases end-to-end
#@markdown - `quick_test`: Short runs (100 steps each) to verify setup
#@markdown - `lang_adapter_only`: Only train lang_rust adapter + merge
#@markdown - `skip_to_rl`: Start from existing core_agent checkpoint (IPO + GRPO only)

gpu_tier = "a100_80gb"  #@param ["a100_40gb", "a100_80gb"]
#@markdown **GPU Tier** (auto-detected below; override here if needed)

max_steps_override = 0  #@param {type:"integer"}
#@markdown Set >0 to cap all training stages at this many steps (0 = use defaults)

skip_data_generation = False  #@param {type:"boolean"}
#@markdown Use pre-generated data from Drive instead of running mutation/trajectory generation

include_grpo = True  #@param {type:"boolean"}
#@markdown GRPO RL is slow; set False to skip

# ============================================================
# GPU TIER CONFIGS
# ============================================================

GPU_CONFIGS = {
    "a100_40gb": {
        "lang_rust": {"batch": 1, "grad_accum": 8, "seq_len": 8192, "max_steps": 3000},
        "core_agent": {"batch": 1, "grad_accum": 4, "seq_len": 12288, "max_steps": 2000},
        "ipo": {"batch": 1, "grad_accum": 8, "seq_len": 12288, "max_steps": 1000},
        "grpo": {"batch": 1, "grad_accum": 4, "seq_len": 16384, "max_steps": 2000, "num_gen": 2},
    },
    "a100_80gb": {
        "lang_rust": {"batch": 1, "grad_accum": 8, "seq_len": 8192, "max_steps": 5000},
        "core_agent": {"batch": 1, "grad_accum": 4, "seq_len": 16384, "max_steps": 3000},
        "ipo": {"batch": 1, "grad_accum": 16, "seq_len": 16384, "max_steps": 2000},
        "grpo": {"batch": 1, "grad_accum": 8, "seq_len": 32768, "max_steps": 5000, "num_gen": 4},
    },
}

# Quick test overrides
if training_scope == "quick_test":
    max_steps_override = 100

gpu_cfg = GPU_CONFIGS[gpu_tier]

# Build CONFIG dict
CONFIG = {
    "training_scope": training_scope,
    "gpu_tier": gpu_tier,
    "include_grpo": include_grpo,
    "skip_data_generation": skip_data_generation,
    # Lang adapter
    "lang_rust_batch": gpu_cfg["lang_rust"]["batch"],
    "lang_rust_grad_accum": gpu_cfg["lang_rust"]["grad_accum"],
    "lang_rust_seq_len": gpu_cfg["lang_rust"]["seq_len"],
    "lang_rust_max_steps": max_steps_override or gpu_cfg["lang_rust"]["max_steps"],
    # Core agent
    "core_agent_batch": gpu_cfg["core_agent"]["batch"],
    "core_agent_grad_accum": gpu_cfg["core_agent"]["grad_accum"],
    "core_agent_seq_len": gpu_cfg["core_agent"]["seq_len"],
    "core_agent_max_steps": max_steps_override or gpu_cfg["core_agent"]["max_steps"],
    # IPO
    "ipo_batch": gpu_cfg["ipo"]["batch"],
    "ipo_grad_accum": gpu_cfg["ipo"]["grad_accum"],
    "ipo_seq_len": gpu_cfg["ipo"]["seq_len"],
    "ipo_max_steps": max_steps_override or gpu_cfg["ipo"]["max_steps"],
    # GRPO
    "grpo_batch": gpu_cfg["grpo"]["batch"],
    "grpo_grad_accum": gpu_cfg["grpo"]["grad_accum"],
    "grpo_seq_len": gpu_cfg["grpo"]["seq_len"],
    "grpo_max_steps": max_steps_override or gpu_cfg["grpo"]["max_steps"],
    "grpo_num_gen": gpu_cfg["grpo"]["num_gen"],
    # Mutation generation
    "max_mutations_per_repo": 50 if training_scope == "quick_test" else 100,
    "mutation_jobs": 4,
    # Eval
    "eval_num_samples": 10 if training_scope == "quick_test" else 50,
}

DRIVE_BASE = "/content/drive/MyDrive/gpt-oss-20b-rust-agent"

print("=" * 60)
print("PIPELINE CONFIGURATION")
print("=" * 60)
print(f"\nScope: {training_scope.upper()}")
print(f"GPU tier: {gpu_tier}")
print(f"Include GRPO: {include_grpo}")
print(f"Skip data gen: {skip_data_generation}")
if max_steps_override:
    print(f"Max steps override: {max_steps_override}")
print(f"\nLang Adapter:  batch={CONFIG['lang_rust_batch']} x grad_accum={CONFIG['lang_rust_grad_accum']}, seq={CONFIG['lang_rust_seq_len']}, steps={CONFIG['lang_rust_max_steps']}")
print(f"Core Agent:    batch={CONFIG['core_agent_batch']} x grad_accum={CONFIG['core_agent_grad_accum']}, seq={CONFIG['core_agent_seq_len']}, steps={CONFIG['core_agent_max_steps']}")
print(f"IPO:           batch={CONFIG['ipo_batch']} x grad_accum={CONFIG['ipo_grad_accum']}, seq={CONFIG['ipo_seq_len']}, steps={CONFIG['ipo_max_steps']}")
if include_grpo:
    print(f"GRPO:          batch={CONFIG['grpo_batch']} x grad_accum={CONFIG['grpo_grad_accum']}, seq={CONFIG['grpo_seq_len']}, steps={CONFIG['grpo_max_steps']}, gen={CONFIG['grpo_num_gen']}")
print("=" * 60)

In [ ]:
#@title ### 0.4 Set Up Persistent Storage

if IN_COLAB:
    print(f"Setting up storage at: {DRIVE_BASE}")

    # Create Drive directories
    drive_dirs = [
        "checkpoints/lang_rust",
        "checkpoints/core_agent",
        "checkpoints/core_agent_ipo",
        "checkpoints/core_agent_grpo",
        "checkpoints/gpt-oss-20b-rust-merged",
        "data/rust/lang_rust",
        "data/rust/core_agent",
        "data/rust/mutations",
        "data/rust/ipo",
        "data/rust/grpo",
        "data/rust/eval",
        "data/rust/repos",
        "logs",
        "evals/rust_agent",
    ]
    for subdir in drive_dirs:
        os.makedirs(os.path.join(DRIVE_BASE, subdir), exist_ok=True)

    # Create symlinks for key directories
    for dir_name in ["checkpoints", "data", "logs", "evals"]:
        local_path = os.path.join(PROJECT_ROOT, dir_name)
        drive_path = os.path.join(DRIVE_BASE, dir_name)

        if os.path.exists(local_path) and not os.path.islink(local_path):
            !cp -r {local_path}/* {drive_path}/ 2>/dev/null || true
            !rm -rf {local_path}
        elif os.path.islink(local_path):
            os.unlink(local_path)

        os.symlink(drive_path, local_path)
        print(f"  {dir_name} -> Drive")

    print("\nStorage ready!")
else:
    for d in ["checkpoints", "data/rust", "logs", "evals/rust_agent"]:
        os.makedirs(d, exist_ok=True)
    print("Local directories created.")

In [ ]:
#@title ### 0.5 Check GPU
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    capability = torch.cuda.get_device_capability()

    CONFIG["use_fp8"] = capability[0] >= 9

    # Auto-detect GPU tier
    detected_tier = "a100_80gb" if gpu_memory >= 70 else "a100_40gb"
    if detected_tier != CONFIG["gpu_tier"]:
        print(f"NOTE: Auto-detected {detected_tier}, overriding configured {CONFIG['gpu_tier']}")
        CONFIG["gpu_tier"] = detected_tier

    print("=" * 60)
    print(f"GPU: {gpu_name} ({gpu_memory:.0f} GB)")
    print(f"Compute capability: {capability[0]}.{capability[1]}")
    print(f"FP8: {'Available' if CONFIG['use_fp8'] else 'Not available'}")
    print(f"Tier: {CONFIG['gpu_tier']}")

    if gpu_memory < 40:
        print("\nWARNING: <40 GB VRAM. GPT-OSS 20B (4-bit) needs ~12 GB for weights")
        print("but long-context training (16K+) may OOM. Consider reducing seq lengths.")
    print("=" * 60)
else:
    print("No GPU detected!")
    CONFIG["use_fp8"] = False

---
## Step 1: Data Generation

Generates mutation data from curated Rust repos and agent trajectories.
Skip this step if you have pre-generated data on Drive (`skip_data_generation=True`).

In [ ]:
#@title ### 1.1 Generate Mutation Data
#@markdown Runs `cargo-mutants` on curated Rust repos to produce bug-fix training pairs.

if CONFIG["skip_data_generation"]:
    print("Skipping data generation (using pre-generated data from Drive)")
else:
    if CONFIG["training_scope"] in ("skip_to_rl",):
        print("Skipping \u2014 not needed for this training scope")
    else:
        max_muts = CONFIG["max_mutations_per_repo"]
        jobs = CONFIG["mutation_jobs"]

        print(f"Generating mutations (max {max_muts}/repo, {jobs} parallel jobs)...")
        print("=" * 60)

        !python scripts/16_generate_mutations.py \
            --max_mutations_per_repo {max_muts} \
            --jobs {jobs}

        # Backup to Drive
        if IN_COLAB:
            drive_mut = os.path.join(DRIVE_BASE, "data/rust/mutations")
            !cp -r data/rust/mutations/* {drive_mut}/ 2>/dev/null || true
            print("\nBacked up mutations to Drive.")

In [ ]:
#@title ### 1.2 Generate Agent Trajectories
#@markdown Generates multi-turn agent trajectories from mutations + Strandset in Harmony format.

if CONFIG["skip_data_generation"]:
    print("Skipping data generation (using pre-generated data from Drive)")
else:
    if CONFIG["training_scope"] in ("skip_to_rl",):
        print("Skipping \u2014 not needed for this training scope")
    else:
        max_samples = 500 if CONFIG["training_scope"] == "quick_test" else 5000

        print(f"Generating trajectories (max {max_samples} per source)...")
        print("=" * 60)

        cmd = f"python scripts/15_generate_trajectories.py --max_samples {max_samples}"

        # Point to mutations if they exist
        mutations_path = "data/rust/mutations/mutations.jsonl"
        if os.path.exists(mutations_path):
            cmd += f" --mutations_path {mutations_path}"

        !{cmd}

        # Backup to Drive
        if IN_COLAB:
            drive_agent = os.path.join(DRIVE_BASE, "data/rust/core_agent")
            !cp -r data/rust/core_agent/* {drive_agent}/ 2>/dev/null || true
            print("\nBacked up trajectories to Drive.")

In [ ]:
#@title ### 1.3 Verify Data

data_checks = [
    ("Mutations", "data/rust/mutations"),
    ("Lang Rust train", "data/rust/lang_rust/train"),
    ("Core Agent train", "data/rust/core_agent/train"),
    ("IPO train", "data/rust/ipo/train"),
    ("GRPO tasks", "data/rust/grpo"),
    ("Eval tasks", "data/rust/eval"),
]

print("Data Verification:")
print("=" * 60)
for name, path in data_checks:
    exists = os.path.exists(path)
    if exists and os.path.isdir(path):
        items = os.listdir(path)
        print(f"  \u2713 {name}: {path} ({len(items)} items)")
    elif exists:
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  \u2713 {name}: {path} ({size_mb:.1f} MB)")
    else:
        needed = True
        if CONFIG["training_scope"] == "skip_to_rl" and name in ("Mutations", "Lang Rust train", "Core Agent train"):
            needed = False
        if CONFIG["training_scope"] == "lang_adapter_only" and name in ("IPO train", "GRPO tasks"):
            needed = False
        sym = "\u2717" if needed else "\u2014"
        label = "MISSING" if needed else "not needed"
        print(f"  {sym} {name}: {label}")
print("=" * 60)

---
## Step 2: Lang Adapter Training

Train a QLoRA adapter (rank 64) to specialise GPT-OSS 20B on Rust syntax, stdlib, and idioms.
Then merge the adapter into the base weights for downstream training.

In [ ]:
#@title ### 2.1 Train lang_rust Adapter
#@markdown Trains a per-language QLoRA adapter on Rust data (rank 64, 1 epoch).

if CONFIG["training_scope"] == "skip_to_rl":
    print("Skipping \u2014 scope is skip_to_rl")
else:
    batch = CONFIG["lang_rust_batch"]
    grad_accum = CONFIG["lang_rust_grad_accum"]
    max_steps = CONFIG["lang_rust_max_steps"]
    seq_len = CONFIG["lang_rust_seq_len"]

    cmd = f"python scripts/13_train_lang_adapter.py"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    print(f"Training lang_rust adapter...")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Seq length: {seq_len} (from config)")
    print("=" * 60)

    !{cmd}

    # Backup checkpoint to Drive
    if IN_COLAB:
        !cp -r checkpoints/lang_rust/* {DRIVE_BASE}/checkpoints/lang_rust/ 2>/dev/null || true
        print("\nCheckpoint backed up to Drive.")

In [ ]:
#@title ### 2.2 Merge lang_rust into Base
#@markdown Merges the lang_rust adapter into the GPT-OSS 20B base weights.

if CONFIG["training_scope"] == "skip_to_rl":
    print("Skipping \u2014 scope is skip_to_rl")
else:
    print("Merging lang_rust adapter into base model...")
    print("=" * 60)

    !python scripts/19_merge_adapter.py \
        --adapter_path checkpoints/lang_rust/final \
        --output_dir checkpoints/gpt-oss-20b-rust-merged \
        --export_formats hf

    # Backup merged model to Drive
    if IN_COLAB:
        !cp -r checkpoints/gpt-oss-20b-rust-merged/* {DRIVE_BASE}/checkpoints/gpt-oss-20b-rust-merged/ 2>/dev/null || true
        print("\nMerged model backed up to Drive.")

In [ ]:
#@title ### 2.3 Verify Merge

if CONFIG["training_scope"] == "skip_to_rl":
    print("Skipping \u2014 scope is skip_to_rl")
else:
    merged_path = "checkpoints/gpt-oss-20b-rust-merged"
    adapter_path = "checkpoints/lang_rust/final"

    print("Merge Verification:")
    print("=" * 60)

    if os.path.exists(merged_path):
        files = os.listdir(merged_path)
        safetensors = [f for f in files if f.endswith(".safetensors")]
        print(f"  \u2713 Merged model: {merged_path}")
        print(f"    {len(safetensors)} safetensors shard(s), {len(files)} total files")
    else:
        print(f"  \u2717 Merged model not found at {merged_path}")

    if os.path.exists(adapter_path):
        adapter_files = os.listdir(adapter_path)
        print(f"  \u2713 Adapter: {adapter_path} ({len(adapter_files)} files)")
    else:
        print(f"  \u2717 Adapter not found at {adapter_path}")

    if CONFIG["training_scope"] == "lang_adapter_only":
        print("\n\u2713 lang_adapter_only scope complete. Stopping here.")

    print("=" * 60)

---
## Step 3: Core Agent SFT

Train a higher-rank LoRA adapter (rank 128) on agent trajectories with tool use.
Uses the merged lang_rust model as the base.

In [ ]:
#@title ### 3.1 Train core_agent Adapter
#@markdown Trains the core agent adapter on Harmony-formatted trajectories (rank 128, 2 epochs).

if CONFIG["training_scope"] in ("lang_adapter_only", "skip_to_rl"):
    print(f"Skipping \u2014 scope is {CONFIG['training_scope']}")
else:
    batch = CONFIG["core_agent_batch"]
    grad_accum = CONFIG["core_agent_grad_accum"]
    max_steps = CONFIG["core_agent_max_steps"]
    seq_len = CONFIG["core_agent_seq_len"]

    cmd = f"python scripts/14_train_core_agent.py"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    print(f"Training core_agent adapter...")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Seq length: {seq_len} (from config)")
    print(f"  LoRA rank: 128")
    print("=" * 60)

    !{cmd}

    # Backup checkpoint to Drive
    if IN_COLAB:
        !cp -r checkpoints/core_agent/* {DRIVE_BASE}/checkpoints/core_agent/ 2>/dev/null || true
        print("\nCheckpoint backed up to Drive.")

In [ ]:
#@title ### 3.2 Verify core_agent

if CONFIG["training_scope"] in ("lang_adapter_only", "skip_to_rl"):
    print(f"Skipping \u2014 scope is {CONFIG['training_scope']}")
else:
    ckpt_path = "checkpoints/core_agent/final"

    print("Core Agent Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  \u2713 Checkpoint: {ckpt_path} ({len(files)} files)")

        # Check adapter config for trainable params
        adapter_config = os.path.join(ckpt_path, "adapter_config.json")
        if os.path.exists(adapter_config):
            import json
            with open(adapter_config) as f:
                cfg = json.load(f)
            print(f"    LoRA rank: {cfg.get('r', '?')}")
            print(f"    Alpha: {cfg.get('lora_alpha', '?')}")
            print(f"    Target modules: {cfg.get('target_modules', '?')}")
    else:
        print(f"  \u2717 Checkpoint not found at {ckpt_path}")

    print("=" * 60)

---
## Step 4: Preference Optimisation (IPO)

Train with Identity Preference Optimisation on ranked pairs.
Very low learning rate (5e-7), 1 epoch only to avoid collapse.

In [ ]:
#@title ### 4.1 Train with IPO
#@markdown IPO training from the core_agent checkpoint. Single epoch, very low LR.

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    batch = CONFIG["ipo_batch"]
    grad_accum = CONFIG["ipo_grad_accum"]
    max_steps = CONFIG["ipo_max_steps"]

    # Determine checkpoint source
    if CONFIG["training_scope"] == "skip_to_rl":
        ipo_checkpoint = "checkpoints/core_agent/final"
        print("Using existing core_agent checkpoint (skip_to_rl mode)")
    else:
        ipo_checkpoint = "checkpoints/core_agent/final"

    cmd = f"python scripts/17_ipo_preference.py"
    cmd += f" --checkpoint {ipo_checkpoint}"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    print(f"Training with IPO...")
    print(f"  Checkpoint: {ipo_checkpoint}")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Loss: IPO (beta=0.1)")
    print("=" * 60)

    !{cmd}

    # Backup checkpoint to Drive
    if IN_COLAB:
        !cp -r checkpoints/core_agent_ipo/* {DRIVE_BASE}/checkpoints/core_agent_ipo/ 2>/dev/null || true
        print("\nCheckpoint backed up to Drive.")

In [ ]:
#@title ### 4.2 Verify IPO

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    ckpt_path = "checkpoints/core_agent_ipo/final"

    print("IPO Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  \u2713 IPO checkpoint: {ckpt_path} ({len(files)} files)")
    else:
        print(f"  \u2717 IPO checkpoint not found at {ckpt_path}")

    # Check tensorboard logs for KL divergence if available
    tb_dir = "checkpoints/core_agent_ipo"
    tb_files = []
    if os.path.exists(tb_dir):
        for root, dirs, fnames in os.walk(tb_dir):
            for fn in fnames:
                if fn.startswith("events.out.tfevents"):
                    tb_files.append(os.path.join(root, fn))
    if tb_files:
        print(f"  \u2713 TensorBoard logs found ({len(tb_files)} event files)")
        print(f"    Monitor KL divergence: warn >0.3, abort >0.5")
    else:
        print(f"  \u2014 No TensorBoard logs found")

    print("=" * 60)

---
## Step 5: GRPO RL

Group Relative Policy Optimisation with execution-based rewards.
Generates N completions per prompt, runs `cargo check/test/clippy`, computes group-relative advantages.

**This step is optional** (`include_grpo=False` to skip).

In [ ]:
#@title ### 5.1 Train with GRPO
#@markdown GRPO RL training from IPO checkpoint. Uses execution rewards.

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
elif not CONFIG["include_grpo"]:
    print("Skipping \u2014 GRPO disabled (include_grpo=False)")
else:
    batch = CONFIG["grpo_batch"]
    grad_accum = CONFIG["grpo_grad_accum"]
    max_steps = CONFIG["grpo_max_steps"]

    grpo_checkpoint = "checkpoints/core_agent_ipo/final"

    cmd = f"python scripts/18_grpo_rl.py"
    cmd += f" --checkpoint {grpo_checkpoint}"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    # On 40GB, disable curriculum (cap at 16K)
    if CONFIG["gpu_tier"] == "a100_40gb":
        print("NOTE: 40GB GPU \u2014 GRPO sequence length capped at 16384")

    print(f"Training with GRPO...")
    print(f"  Checkpoint: {grpo_checkpoint}")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Generations per prompt: {CONFIG['grpo_num_gen']}")
    print("=" * 60)

    !{cmd}

    # Backup checkpoint to Drive
    if IN_COLAB:
        !cp -r checkpoints/core_agent_grpo/* {DRIVE_BASE}/checkpoints/core_agent_grpo/ 2>/dev/null || true
        print("\nCheckpoint backed up to Drive.")

In [ ]:
#@title ### 5.2 Verify GRPO

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
elif not CONFIG["include_grpo"]:
    print("Skipping \u2014 GRPO disabled")
else:
    ckpt_path = "checkpoints/core_agent_grpo/final"

    print("GRPO Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  \u2713 GRPO checkpoint: {ckpt_path} ({len(files)} files)")
    else:
        print(f"  \u2717 GRPO checkpoint not found at {ckpt_path}")

    print("=" * 60)

---
## Step 6: Evaluation

Evaluate the best checkpoint on held-out Rust tasks using execution-based metrics
(cargo check, cargo test, clippy).

In [ ]:
#@title ### 6.1 Run Rust Evaluation
#@markdown Evaluates the best checkpoint on held-out Rust coding tasks.

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    # Determine best checkpoint
    if CONFIG["include_grpo"] and os.path.exists("checkpoints/core_agent_grpo/final"):
        eval_checkpoint = "checkpoints/core_agent_grpo/final"
    elif os.path.exists("checkpoints/core_agent_ipo/final"):
        eval_checkpoint = "checkpoints/core_agent_ipo/final"
    elif os.path.exists("checkpoints/core_agent/final"):
        eval_checkpoint = "checkpoints/core_agent/final"
    else:
        eval_checkpoint = "checkpoints/core_agent_ipo/final"  # fallback

    num_samples = CONFIG["eval_num_samples"]

    print(f"Evaluating checkpoint: {eval_checkpoint}")
    print(f"Samples: {num_samples}")
    print("=" * 60)

    !python scripts/eval_rust_agent.py \
        --checkpoint {eval_checkpoint} \
        --num_samples {num_samples}

    # Backup results to Drive
    if IN_COLAB:
        !cp -r evals/rust_agent/* {DRIVE_BASE}/evals/rust_agent/ 2>/dev/null || true
        print("\nResults backed up to Drive.")

In [ ]:
#@title ### 6.2 Check Promotion Gates
#@markdown Checks whether the model meets the promotion thresholds.

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    print("Checking promotion gates...")
    print("=" * 60)

    !python scripts/12_check_gates.py rust_agent

In [ ]:
#@title ### 6.3 Display Results
#@markdown Loads and pretty-prints evaluation metrics.

if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    import json

    metrics_path = "evals/rust_agent/metrics.json"

    if os.path.exists(metrics_path):
        with open(metrics_path) as f:
            metrics = json.load(f)

        # Target thresholds from configs/rust_eval.yaml
        targets = {
            "cargo_check_pass_rate": (0.85, "higher"),
            "cargo_test_pass_rate": (0.70, "higher"),
            "clippy_clean_rate": (0.80, "higher"),
            "iterations_to_green_median": (3, "lower"),
            "diff_size_median": (50, "lower"),
            "tool_call_format_accuracy": (0.99, "higher"),
            "hallucinated_api_rate": (0.05, "lower"),
        }

        print("=" * 60)
        print("EVALUATION RESULTS")
        print("=" * 60)
        print(f"{'Metric':<32} {'Value':>8} {'Target':>8} {'Status':>8}")
        print("-" * 60)

        for key, (target, direction) in targets.items():
            value = metrics.get(key)
            if value is None:
                print(f"{key:<32} {'N/A':>8} {target:>8} {'\u2014':>8}")
                continue

            if direction == "higher":
                passed = value >= target
            else:
                passed = value <= target

            status = "\u2713 PASS" if passed else "\u2717 FAIL"
            fmt_val = f"{value:.2%}" if isinstance(value, float) and value <= 1 else f"{value}"
            fmt_tgt = f"{target:.0%}" if isinstance(target, float) and target <= 1 else f"{target}"
            print(f"{key:<32} {fmt_val:>8} {fmt_tgt:>8} {status:>8}")

        print("=" * 60)
    else:
        print(f"\u2717 Metrics file not found at {metrics_path}")
        print("Run evaluation (6.1) first.")

---
## Step 7: Test Model

Load the trained model and generate Rust code interactively.

In [ ]:
#@title ### 7.1 Load Model
#@markdown Loads the best checkpoint via Unsloth for fast inference.

from unsloth import FastLanguageModel
import torch

# Determine best checkpoint
CHECKPOINT_PRIORITY = [
    "checkpoints/core_agent_grpo/final",
    "checkpoints/core_agent_ipo/final",
    "checkpoints/core_agent/final",
    "checkpoints/gpt-oss-20b-rust-merged",
]

MODEL_PATH = None
for path in CHECKPOINT_PRIORITY:
    if os.path.exists(path):
        MODEL_PATH = path
        break

if MODEL_PATH is None:
    print("\u2717 No checkpoint found. Train the model first.")
else:
    print(f"Loading model from: {MODEL_PATH}")
    print("=" * 60)

    model, tokenizer = FastLanguageModel.from_pretrained(
        MODEL_PATH,
        max_seq_length=4096,
        load_in_4bit=True,
        dtype=torch.bfloat16,
    )
    FastLanguageModel.for_inference(model)

    print("\u2713 Model loaded!")

In [ ]:
#@title ### 7.2 Generate Rust Code
#@markdown Tests the model on 3 pre-defined Rust prompts using Harmony format.

import sys
sys.path.insert(0, "scripts")
from dataset_formatters.harmony import encode_harmony_messages

TEST_PROMPTS = [
    "Write a Rust function `fn merge_sorted(a: &[i32], b: &[i32]) -> Vec<i32>` that merges two sorted slices into a single sorted vector.",
    "This Rust code fails the borrow checker. Fix it:\n```rust\nfn main() {\n    let mut v = vec![1, 2, 3];\n    let first = &v[0];\n    v.push(4);\n    println!(\"{}\", first);\n}\n```",
    "Write an async Rust function using tokio that fetches a URL with reqwest, retries up to 3 times on failure, and returns the response body as a String.",
]

def generate_rust(prompt, max_tokens=1024):
    messages = [{"role": "user", "content": prompt}]
    formatted = encode_harmony_messages(
        messages,
        developer_instructions="You are a Rust programming expert. Write correct, idiomatic code.",
    )
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
        )
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

for i, prompt in enumerate(TEST_PROMPTS, 1):
    print(f"\n{'=' * 60}")
    print(f"Test {i}: {prompt[:80]}...")
    print("=" * 60)
    response = generate_rust(prompt)
    print(response)
    print()

In [ ]:
#@title ### 7.3 Custom Prompt

CUSTOM_PROMPT = "Write a Rust function that reads a CSV file and returns the sum of a specified column."  #@param {type:"string"}

print(f"Prompt: {CUSTOM_PROMPT}")
print("=" * 60)
print(generate_rust(CUSTOM_PROMPT))

---
## Step 8: Export

Merge the final adapter and export to HuggingFace + GGUF formats.

In [ ]:
#@title ### 8.1 Export to GGUF
#@markdown Merges the best adapter into the base model and exports as HF safetensors + GGUF Q4.

# Find best adapter
ADAPTER_PRIORITY = [
    "checkpoints/core_agent_grpo/final",
    "checkpoints/core_agent_ipo/final",
    "checkpoints/core_agent/final",
    "checkpoints/lang_rust/final",
]

adapter_path = None
for path in ADAPTER_PRIORITY:
    if os.path.exists(path):
        adapter_path = path
        break

if adapter_path is None:
    print("\u2717 No adapter checkpoint found.")
else:
    export_dir = "checkpoints/gpt-oss-20b-rust-export"
    print(f"Exporting adapter: {adapter_path}")
    print(f"Output: {export_dir}")
    print("=" * 60)

    !python scripts/19_merge_adapter.py \
        --adapter_path {adapter_path} \
        --output_dir {export_dir} \
        --export_formats hf gguf_q4

    # Backup to Drive
    if IN_COLAB:
        drive_export = os.path.join(DRIVE_BASE, "checkpoints/gpt-oss-20b-rust-export")
        os.makedirs(drive_export, exist_ok=True)
        !cp -r {export_dir}/* {drive_export}/ 2>/dev/null || true
        print("\nExport backed up to Drive.")

In [ ]:
#@title ### 8.2 Download GGUF
#@markdown Downloads the GGUF file directly (Colab only).

if IN_COLAB:
    from google.colab import files
    import glob

    export_dir = "checkpoints/gpt-oss-20b-rust-export"
    gguf_files = glob.glob(os.path.join(export_dir, "*.gguf"))

    if gguf_files:
        gguf_path = gguf_files[0]
        size_gb = os.path.getsize(gguf_path) / (1024**3)
        print(f"Downloading: {os.path.basename(gguf_path)} ({size_gb:.1f} GB)")
        files.download(gguf_path)
    else:
        print("\u2717 No GGUF file found. Run export (8.1) first.")
else:
    print("Download not available outside Colab.")
    print("GGUF file is at: checkpoints/gpt-oss-20b-rust-export/")

---
## Training Complete!

Your GPT-OSS 20B Rust coding agent is trained and ready to use.

**Outputs:**
- Checkpoints: `checkpoints/core_agent_{ipo,grpo}/final`
- Evaluation: `evals/rust_agent/metrics.json`
- Exported model: `checkpoints/gpt-oss-20b-rust-export/`
- All backed up to Google Drive: `{DRIVE_BASE}`

**Next steps:**
- Review evaluation metrics in Step 6.3
- Test interactively in Step 7
- Deploy the GGUF file with llama.cpp or Ollama